In [1]:
%cd /kaggle/working/

/kaggle/working


In [2]:
!git clone https://github.com/VarunGumma/IndicTransTokenizer

Cloning into 'IndicTransTokenizer'...
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 123 (delta 52), reused 94 (delta 31), pack-reused 0
Receiving objects: 100% (123/123), 3.85 MiB | 15.72 MiB/s, done.
Resolving deltas: 100% (52/52), done.


In [3]:
%cd /kaggle/working/IndicTransTokenizer

/kaggle/working/IndicTransTokenizer


In [4]:
pip install --editable ./

Obtaining file:///kaggle/working/IndicTransTokenizer
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/VarunGumma/indic_nlp_library to /tmp/pip-install-p44fobw9/indic-nlp-library-it2_964d394ed49f4eceabad4700940e3679
  Running command git clone --filter=blob:none --quiet https://github.com/VarunGumma/indic_nlp_library /tmp/pip-install-p44fobw9/indic-nlp-library-it2_964d394ed49f4eceabad4700940e3679
  Resolved https://github.com/VarunGumma/indic_nlp_library to commit 09d30a15286cc252a12682e5450c807379717eaf
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 931.0 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 2.2 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.9/807.9 kB 9.1 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 48.7 MB/s eta 0:00:00a 0:

In [5]:
import torch
from transformers import AutoModelForSeq2SeqLM
from IndicTransTokenizer import IndicProcessor, IndicTransTokenizer

## SELECT RANDOM 1000 LINES

In [6]:
with open('/kaggle/input/selected/enselected.txt', 'r') as f1:
    selected_en = f1.readlines()
    
with open('/kaggle/input/selected/hiselected.txt', 'r') as f1:
    selected_hi = f1.readlines()
    
with open('/kaggle/input/selected/taselected.txt', 'r') as f1:
    selected_ta = f1.readlines()

## ENGLISH TO HINDI 

In [10]:
tokenizer = IndicTransTokenizer(direction="en-indic")
ip = IndicProcessor(inference=True)
model = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/indictrans2-en-indic-dist-200M", trust_remote_code=True,low_cpu_mem_usage=True).to("cuda")

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

configuration_indictrans.py:   0%|          | 0.00/14.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-dist-200M:
- configuration_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_indictrans.py:   0%|          | 0.00/61.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-dist-200M:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

In [12]:
answers = []
BATCH_SIZE = 4
for i in range(0,len(selected_en),BATCH_SIZE):
    batch = selected_en[i : i + BATCH_SIZE]

    # Preprocess the batch and extract entity mappings
    batch = ip.preprocess_batch(batch, src_lang="eng_Latn", tgt_lang="hin_Deva")

    # Tokenize the batch and generate input encodings
    batch = tokenizer(batch, src=True, return_tensors="pt").to("cuda")

    with torch.inference_mode():
        outputs = model.generate(**batch,use_cache=True, num_beams=5, num_return_sequences=1, max_length=256)

    # Decode the generated tokens into text
    outputs = tokenizer.batch_decode(outputs, src=False)

    # Postprocess the translations, including entity replacement
    outputs = ip.postprocess_batch(outputs, lang="hin_Deva")
    answers.append(outputs)
    del batch
    torch.cuda.empty_cache()
print(len(answers))

250


In [15]:
output_file = '/kaggle/working/ITen_to_hi.txt'
with open(output_file, 'w') as f:
    for lst in answers:
        for line in lst:
            f.write(line+"\n")

## HINDI TO ENGLISH

In [16]:
tokenizer = IndicTransTokenizer(direction="indic-en")
ip = IndicProcessor(inference=True)
model = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/indictrans2-indic-en-dist-200M", trust_remote_code=True, low_cpu_mem_usage=True).to("cuda")

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

configuration_indictrans.py:   0%|          | 0.00/14.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-en-dist-200M:
- configuration_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_indictrans.py:   0%|          | 0.00/61.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-en-dist-200M:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/914M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

In [19]:
answers = []
BATCH_SIZE = 4
for i in range(0,len(selected_hi),BATCH_SIZE):
    batch = selected_hi[i : i + BATCH_SIZE]

    # Preprocess the batch and extract entity mappings
    batch = ip.preprocess_batch(batch, src_lang="hin_Deva", tgt_lang="eng_Latn")

    # Tokenize the batch and generate input encodings
    batch = tokenizer(batch, src=True, return_tensors="pt").to("cuda")

    with torch.inference_mode():
        outputs = model.generate(**batch,use_cache=True, num_beams=5, num_return_sequences=1, max_length=256)

    # Decode the generated tokens into text
    outputs = tokenizer.batch_decode(outputs, src=False)

    # Postprocess the translations, including entity replacement
    outputs = ip.postprocess_batch(outputs, lang="eng_Latn")
    answers.append(outputs)
    del batch
    torch.cuda.empty_cache()
print(len(answers))

250


In [20]:
output_file = '/kaggle/working/IThi_to_en.txt'
with open(output_file, 'w') as f:
    for lst in answers:
        for line in lst:
            f.write(line+"\n")

## TAMIL TO HINDI

In [21]:
tokenizer = IndicTransTokenizer(direction="indic-indic")
ip = IndicProcessor(inference=True)
model = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/indictrans2-indic-indic-dist-320M", trust_remote_code=True, low_cpu_mem_usage=True).to("cuda")

config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

configuration_indictrans.py:   0%|          | 0.00/14.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-indic-dist-320M:
- configuration_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_indictrans.py:   0%|          | 0.00/61.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-indic-dist-320M:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/1.28G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [22]:
answers = []
BATCH_SIZE = 4
for i in range(0,len(selected_ta),BATCH_SIZE):
    batch = selected_ta[i : i + BATCH_SIZE]

    # Preprocess the batch and extract entity mappings
    batch = ip.preprocess_batch(batch, src_lang="tam_Taml", tgt_lang="hin_Deva")

    # Tokenize the batch and generate input encodings
    batch = tokenizer(batch, src=True, return_tensors="pt").to("cuda")

    with torch.inference_mode():
        outputs = model.generate(**batch,use_cache=True, num_beams=5, num_return_sequences=1, max_length=256)

    # Decode the generated tokens into text
    outputs = tokenizer.batch_decode(outputs, src=False)

    # Postprocess the translations, including entity replacement
    outputs = ip.postprocess_batch(outputs, lang="hin_Deva")
    answers.append(outputs)
    del batch
    torch.cuda.empty_cache()
print(len(answers))

250


In [23]:
output_file = '/kaggle/working/ITta_to_hi.txt'
with open(output_file, 'w') as f:
    for lst in answers:
        for line in lst:
            f.write(line+"\n")

## HINDI TO TAMIL

In [24]:
answers = []
BATCH_SIZE = 4
for i in range(0,len(selected_hi),BATCH_SIZE):
    batch = selected_hi[i : i + BATCH_SIZE]

    # Preprocess the batch and extract entity mappings
    batch = ip.preprocess_batch(batch, src_lang="hin_Deva", tgt_lang="tam_Taml")

    # Tokenize the batch and generate input encodings
    batch = tokenizer(batch, src=True, return_tensors="pt").to("cuda")

    with torch.inference_mode():
        outputs = model.generate(**batch,use_cache=True, num_beams=5, num_return_sequences=1, max_length=256)

    # Decode the generated tokens into text
    outputs = tokenizer.batch_decode(outputs, src=False)

    # Postprocess the translations, including entity replacement
    outputs = ip.postprocess_batch(outputs, lang="tam_Taml")
    answers.append(outputs)
    del batch
    torch.cuda.empty_cache()
print(len(answers))

250


In [25]:
output_file = '/kaggle/working/IThi_to_ta.txt'
with open(output_file, 'w') as f:
    for lst in answers:
        for line in lst:
            f.write(line+"\n")

## BLEU SCORE

In [1]:
with open('/kaggle/input/selected/enselected.txt', 'r') as f1:
    en = f1.readlines()

with open('/kaggle/input/selected/hiselected.txt', 'r') as f2:
    hi = f2.readlines()

with open('/kaggle/input/selected/taselected.txt', 'r') as f2:
    ta = f2.readlines()

In [2]:
with open('/kaggle/input/outputs/ITen_to_hi.txt', 'r') as f1:
    en_to_hi = f1.readlines()
    
with open('/kaggle/input/outputs/IThi_to_en.txt', 'r') as f1:
    hi_to_en = f1.readlines()

with open('/kaggle/input/outputs/ITta_to_hi.txt', 'r') as f1:
    ta_to_hi = f1.readlines()
    
with open('/kaggle/input/outputs/IThi_to_ta.txt', 'r') as f1:
    hi_to_ta = f1.readlines()

In [4]:
from nltk.translate.bleu_score import corpus_bleu

In [5]:
bleu_score = corpus_bleu(en[:50], hi_to_en[:50])
print("BLEU SCORE FOR HINDI TO ENGLISH: ",bleu_score)

BLEU SCORE FOR HINDI TO ENGLISH:  0.6946380319226029


/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [6]:
bleu_score = corpus_bleu(hi[:50], en_to_hi[:50])
print("BLEU SCORE FOR ENGLISH TO HINDI: ",bleu_score)

BLEU SCORE FOR ENGLISH TO HINDI:  0.7393176589394633


In [7]:
bleu_score = corpus_bleu(hi[:50], ta_to_hi[:50])
print("BLEU SCORE FOR TAMIL TO HINDI: ",bleu_score)

BLEU SCORE FOR TAMIL TO HINDI:  0.729996495977605


In [8]:
bleu_score = corpus_bleu(ta[:50], hi_to_ta[:50])
print("BLEU SCORE FOR HINDI TO TAMIL: ",bleu_score)

BLEU SCORE FOR HINDI TO TAMIL:  0.6928307685071132


## ROUGE SCORE

In [9]:
!pip install rouge

In [10]:
from rouge import Rouge
rouge = Rouge()

In [11]:
scores = rouge.get_scores(hi_to_en[:50], en[:50], avg=True)
print("ROUGE SCORE FOR HINDI TO ENGLISH: ")
for key,value in scores.items():
    print(key, ":" ,value)

ROUGE SCORE FOR HINDI TO ENGLISH: 
rouge-1 : {'r': 0.6611118471653776, 'p': 0.6520790173832468, 'f': 0.6531700197799464}
rouge-2 : {'r': 0.4205068189613888, 'p': 0.4193624139560719, 'f': 0.4173564109556508}
rouge-l : {'r': 0.6260610716086846, 'p': 0.6166978733038269, 'f': 0.6181932888088356}


In [12]:
scores = rouge.get_scores(en_to_hi[:50], hi[:50], avg=True)
print("ROUGE SCORE FOR ENGLISH TO HINDI: ")
for key,value in scores.items():
    print(key, ":" ,value)

ROUGE SCORE FOR ENGLISH TO HINDI: 
rouge-1 : {'r': 0.5895219500982772, 'p': 0.5909189053298693, 'f': 0.5870490708000117}
rouge-2 : {'r': 0.34807683542135004, 'p': 0.35522161759378307, 'f': 0.3494625915233706}
rouge-l : {'r': 0.5529000755287315, 'p': 0.555578366927625, 'f': 0.5512373093550673}


In [13]:
scores = rouge.get_scores(ta_to_hi[:50], hi[:50], avg=True)
print("ROUGE SCORE FOR TAMIL TO HINDI: ")
for key,value in scores.items():
    print(key, ":" ,value)

ROUGE SCORE FOR TAMIL TO HINDI: 
rouge-1 : {'r': 0.44568356454458813, 'p': 0.4424566099149152, 'f': 0.43982184817519515}
rouge-2 : {'r': 0.2108982417572706, 'p': 0.2109436988595825, 'f': 0.20848058396385358}
rouge-l : {'r': 0.40959540379466264, 'p': 0.4089512651876024, 'f': 0.40550507700485044}


In [14]:
scores = rouge.get_scores(hi_to_ta[:50], ta[:50], avg=True)
print("ROUGE SCORE FOR HINDI TO TAMIL: ")
for key,value in scores.items():
    print(key, ":" ,value)

ROUGE SCORE FOR HINDI TO TAMIL: 
rouge-1 : {'r': 0.4002495094084237, 'p': 0.4054384804409033, 'f': 0.3995532037804487}
rouge-2 : {'r': 0.1506372714832467, 'p': 0.14701055537244795, 'f': 0.14767167917612337}
rouge-l : {'r': 0.3814232407931084, 'p': 0.38509125456736176, 'f': 0.38019024751203057}
